In [15]:
import torch
from torchvision import datasets, transforms as T
import torchvision
import torch.nn as nn
import torchvision.models as models

In [16]:
transforms = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [17]:

train_dataset = datasets.CIFAR10(
    root="./data",
    download=True,
    train=True,
    transform=transforms,
)
test_dataset = datasets.CIFAR10(
    root="./data",
    download=True,
    train=False,
    transform=transforms
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=64
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=True,
    batch_size=64
)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
model = models.googlenet(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
model

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
from torchsummary import summary

model.to(device)

summary(model, (3, 229, 229), 64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 115, 115]           9,408
       BatchNorm2d-2         [64, 64, 115, 115]             128
       BasicConv2d-3         [64, 64, 115, 115]               0
         MaxPool2d-4           [64, 64, 57, 57]               0
            Conv2d-5           [64, 64, 57, 57]           4,096
       BatchNorm2d-6           [64, 64, 57, 57]             128
       BasicConv2d-7           [64, 64, 57, 57]               0
            Conv2d-8          [64, 192, 57, 57]         110,592
       BatchNorm2d-9          [64, 192, 57, 57]             384
      BasicConv2d-10          [64, 192, 57, 57]               0
        MaxPool2d-11          [64, 192, 28, 28]               0
           Conv2d-12           [64, 64, 28, 28]          12,288
      BatchNorm2d-13           [64, 64, 28, 28]             128
      BasicConv2d-14           [64, 64,

In [22]:
model.fc

Linear(in_features=1024, out_features=1000, bias=True)

In [23]:
model.fc = nn.Linear(1024, 10)

In [24]:
for name, param in model.named_parameters():
  if "inception5" in name or "fc" in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

In [25]:
model.to(device)
summary(model, (3, 229, 229), 64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 115, 115]           9,408
       BatchNorm2d-2         [64, 64, 115, 115]             128
       BasicConv2d-3         [64, 64, 115, 115]               0
         MaxPool2d-4           [64, 64, 57, 57]               0
            Conv2d-5           [64, 64, 57, 57]           4,096
       BatchNorm2d-6           [64, 64, 57, 57]             128
       BasicConv2d-7           [64, 64, 57, 57]               0
            Conv2d-8          [64, 192, 57, 57]         110,592
       BatchNorm2d-9          [64, 192, 57, 57]             384
      BasicConv2d-10          [64, 192, 57, 57]               0
        MaxPool2d-11          [64, 192, 28, 28]               0
           Conv2d-12           [64, 64, 28, 28]          12,288
      BatchNorm2d-13           [64, 64, 28, 28]             128
      BasicConv2d-14           [64, 64,

In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

In [29]:
EPOCHS = 10
from tqdm import tqdm

for epoch in tqdm(range(EPOCHS)):
  for i, (images, labels ) in enumerate(train_dataloader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)

    loss = loss_fn(outputs, labels)
    predicted_labels = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
    accuracy = (predicted_labels==labels).sum().item()/len(outputs)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 300 == 0:
      print(f"Epoch: {epoch+1}/{EPOCHS}, Step: {i+1}/{len(train_dataloader)}")
      print(f"Train Accuracy: {accuracy}, Train Loss: {loss}")
      print("\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1/10, Step: 300/782
Train Accuracy: 0.734375, Train Loss: 0.8119601607322693


Epoch: 1/10, Step: 600/782
Train Accuracy: 0.875, Train Loss: 0.42532625794410706




 10%|█         | 1/10 [04:40<42:01, 280.18s/it]

Epoch: 2/10, Step: 300/782
Train Accuracy: 0.859375, Train Loss: 0.38350242376327515


Epoch: 2/10, Step: 600/782
Train Accuracy: 0.8125, Train Loss: 0.49523845314979553




 20%|██        | 2/10 [09:06<36:14, 271.77s/it]

...

 80%|████████  | 8/10 [35:37<08:50, 265.47s/it]

Epoch: 9/10, Step: 300/782
Train Accuracy: 0.90625, Train Loss: 0.23313665390014648


Epoch: 9/10, Step: 600/782
Train Accuracy: 0.890625, Train Loss: 0.2754126489162445




 90%|█████████ | 9/10 [40:01<04:25, 265.16s/it]

Epoch: 10/10, Step: 300/782
Train Accuracy: 0.953125, Train Loss: 0.14523135125637054


Epoch: 10/10, Step: 600/782
Train Accuracy: 0.875, Train Loss: 0.3345590829849243




100%|██████████| 10/10 [44:26<00:00, 266.67s/it]


In [31]:
test_loss, test_acc = 0, 0

with torch.inference_mode():
  for (images, labels) in test_dataloader:
    model.eval()
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)

    loss = loss_fn(outputs, labels)

    predicted_labels = torch.argmax(torch.softmax(outputs, dim=1), dim=1)

    accuracy = (predicted_labels == labels).sum().item()/len(outputs)
    test_acc += accuracy
    test_loss += loss

  test_acc /= len(test_dataloader)
  test_loss /= len(test_dataloader)
  print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

Test Accuracy: 0.8774880573248408, Test Loss: 0.3708511292934418
